## Epoll in Python
epoll是linux内核的io多路复用的一种机制, 可以监视多个fd的读/写等事件, 一旦某个描述符就绪（一般是读或者写事件发生了），就能够将发生的事件通知给关心的应用程序去处理该事件    

### io多路复用的本质
Linux通过socket睡眠队列来管理所有等待socket的某个事件的process，同时通过wakeup机制来异步唤醒整个睡眠队列上等待事件的process，通知process相关事件发生。通常情况，socket的事件发生的时候，其会顺序遍历socket睡眠队列上的每个process节点，调用每个process节点挂载的callback函数。在遍历的过程中，如果遇到某个节点是排他的，那么就终止遍历，总体上会涉及两大逻辑：
    1. 睡眠等待逻辑      
    2. 唤醒逻辑    
    
#### 睡眠等待逻辑 (阻塞)
1. select、poll、epoll_wait陷入内核，判断监控的socket是否有关心的事件发生了，如果没，则为当前process构建一个wait_entry节点，然后插入到监控socket的sleep_list
2. 直到完成队列的遍历或遇到某个wait_entry节点是排他的才停止。
3. 关心的事件发生后，将当前process的wait_entry节点从socket的sleep_list中删除。

#### 唤醒逻辑
1. socket的事件发生了，然后socket顺序遍历其睡眠队列，依次调用每个wait_entry节点的callback函数.
2. 直到完成队列的遍历或遇到某个wait_entry节点是排他的才停止.
3. 一般情况下callback包含两个逻辑：     
    - wait_entry自定义的私有逻辑         
    - 唤醒的公共逻辑，主要用于将该wait_entry的process放入CPU的就绪队列，让CPU随后可以调度其执行。
***
**select对于上面两个逻辑的处理:**    
1. select为每个socket引入一个poll逻辑，该poll逻辑用于收集socket发生的事件
2. 当用户porcess调用select的时候， select会将需要监控的fds集合拷贝到内核空间，然后遍历自己监控的socket，调用poll逻辑，是否有事件就绪。如果没有就绪事件，process进入休眠。如果timeout事件内有就绪事件，或者等待timeout了，select会再次遍历监控的socket，挨个收集就绪事件。

存在的问题：
- 被监控的fds需要从用户空间拷贝到内核空间，为了减少数据拷贝带来的性能损坏，内核对被监控的fds集合大小做了限制，并且这个是通过宏控制的，大小不可改变(限制为1024)。
- 被监控的fds集合中，只要有一个事件触发，整个socket集合就会被遍历一次调用socketk的poll函数收集事件。

(poll和select很像， poll只是解决了select对fds大小1024限制问题)
***
**epoll的处理:**       
对于IO复用，需要进行下面的处理:
   - 准备好需要监控的fds集合
   - 探测并返回fds集合中哪些fd可读

1. epoll通过内核与用户空间mmap(内存映射)同一块内存来解决，减少用户态与内核态之间的数据交换
2. epoll通过epoll_ctl来对监控的fds集合来进行增、删、改， 将高频调用的epoll_wait和低频的epoll_ctl隔离开。
3. epoll引入了一个中间层，一个双向链表(ready_list)，通过回调将已经就绪的fd插入ready_list       

LT(level Triggered 水平触发) 有数据存在缓冲区， 则事件一直触发      
ET(Edge Triggered 边沿触发): 仅在缓冲区状态变化时触发事件，比如数据缓冲去从无到有的时候(不可读-可读)
***   


优秀参考资料:         
[大话 Select、Poll、Epoll](https://cloud.tencent.com/developer/article/1005481)      
[How to ues linux epoll with python](http://scotdoyle.com/python-epoll-howto.html)



**设计一个epoll事件驱动的程序一般有以下几个步骤:**     
1. 申请一个epoll对象
2. 告诉epoll对象监视指定socket上的事件
3. 轮询epoll对象，看那些socket上的事件就绪了
4. socket执行对应的逻辑
5. 告诉epoll对象修改要监视的套接字和/或事件的列表
6. 循环3-5直到结束
7. 销毁epoll对象

In [ ]:
import socket, select

EOL1 = b'\n\n'
EOL2 = b'\n\r\n'
response  = b'HTTP/1.0 200 OK\r\nDate: Mon, 1 Jan 1996 01:01:01 GMT\r\n'
response += b'Content-Type: text/plain\r\nContent-Length: 13\r\n\r\n'
response += b'Hello, world!'

serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serversocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
serversocket.bind(('0.0.0.0', 8080))
serversocket.listen(1)
serversocket.setblocking(0)

# 申请epoll对象
epoll = select.epoll()

# 注册socket， 事件给epoll对象
epoll.register(serversocket.fileno(), select.EPOLLIN)

try:
   connections = {}; requests = {}; responses = {}
   # 轮询epoll对象看那些socket，event就绪
   while True:
      events = epoll.poll(1)
      for fileno, event in events:
         if fileno == serversocket.fileno():
            connection, address = serversocket.accept()
            connection.setblocking(0)
            epoll.register(connection.fileno(), select.EPOLLIN)
            connections[connection.fileno()] = connection
            requests[connection.fileno()] = b''
            responses[connection.fileno()] = response
         elif event & select.EPOLLIN:
            requests[fileno] += connections[fileno].recv(1024)
            if EOL1 in requests[fileno] or EOL2 in requests[fileno]:
               epoll.modify(fileno, select.EPOLLOUT)
               print('-'*40 + '\n' + requests[fileno].decode()[:-2])
         elif event & select.EPOLLOUT:
            byteswritten = connections[fileno].send(responses[fileno])
            responses[fileno] = responses[fileno][byteswritten:]
            if len(responses[fileno]) == 0:
               epoll.modify(fileno, 0)
               connections[fileno].shutdown(socket.SHUT_RDWR)
         elif event & select.EPOLLHUP:
            epoll.unregister(fileno)
            connections[fileno].close()
            del connections[fileno]
finally:
   epoll.unregister(serversocket.fileno())
   epoll.close()
   serversocket.close()